In [46]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://tw.stock.yahoo.com/quote/2303.TW/"

response = requests.get(url)

soup = bs(response.text, "lxml")

# print(soup)

In [ ]:
# 取得股票名稱 HTML 元素
stock_name_element = soup.find("h1", class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)")
print(stock_name_element)

# 取得股票名稱 HTML 元素的文字內容
stock_name = stock_name_element.text
print(stock_name)

'''
因為股票漲跌時使用的 class 會不同
所以無法直接用固定的 class 名稱去抓取股票價格的元素
如果確認第一個使用 class 為 Fz(32px) 的 span 元素就是股票價格，可以只用這個 class 名稱去抓取
'''
# stock_price_element = soup.find('span', class_='Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)')
# stock_price_element = soup.find('span', class_='Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)')
stock_price_element = soup.find('span', class_='Fz(32px)')
print(stock_price_element)

stock_price = stock_price_element.text
print(stock_price)


<h1 class="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)">聯電</h1>
聯電
<span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)">45.60</span>
45.60


In [ ]:
# 解決辦法1 - 先找到父元素
stock_price_parent_element = soup.find('div', class_='D(f) Ai(fe) Mb(4px)')
print(stock_price_parent_element)

# 使用 next_element 取得下一個元素
print(stock_price_parent_element.next_element.text)
# 使用 find() 找到指定子元素
print(stock_price_parent_element.find("span").text)

<div class="D(f) Ai(fe) Mb(4px)"><span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)">45.60</span><span class="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-down)"><span class="Mend(4px) Bds(s)" style="border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px"></span>0.40</span><span class="Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c) C($c-trend-down)">(0.87%)</span></div>
45.60
45.60


In [19]:
# 解決辦法 2 - 使用 re.compile()
import re

# 使用 re.compile() 遇到 () 需要用 \( 和 \) 來表示
stock_price_element = soup.find('span', class_=re.compile(r'Fz\(32px\) Fw\(b\) Lh\(1\) Mend\(16px\) D\(f\) Ai\(c\)'))
stock_price = stock_price_element.text
print(stock_price)


42.37


In [ ]:
# 解決辦法 3 - 使用 select_one()
# CSS 要用 . 來分隔每個 class
# stock_price_element = soup.select_one('span.Fz\\(32px\\).Fw\\(b\\).Lh\\(1\\).Mend\\(16px\\).D\\(f\\).Ai\\(c\\)')
# stock_price_element = soup.select_one('span.Fz\\(32px\\)')

stock_price = stock_price_element.text
print(stock_price)


37.47


In [36]:
import re
# 打包成 Function
def get_stock_price(code):
    url = f"https://tw.stock.yahoo.com/quote/{code}.TW/"
    response = requests.get(url)
    soup = bs(response.text, "lxml")

    stock_name_element = soup.find("h1", class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)")
    stock_name = stock_name_element.text
    
    stock_price_element = soup.find('span', class_=re.compile(r'Fz\(32px\) Fw\(b\) Lh\(1\) Mend\(16px\) D\(f\) Ai\(c\)'))
    stock_price = stock_price_element.text

    return stock_name, stock_price

stock_name, stock_price = get_stock_price("0050")
print(f"{stock_name}: {stock_price}")
stock_name, stock_price = get_stock_price("0056")
print(f"{stock_name}: {stock_price}")
stock_name, stock_price = get_stock_price("2330")
print(f"{stock_name}: {stock_price}")

元大台灣50: 64.40
元大高股息: 37.47
台積電: 1,505


In [ ]:
'''
課堂練習
多抓取漲跌資訊
'''